In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore')
import missingno
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.metrics import accuracy_score

from matplotlib import font_manager, rc
font_path = "/content/drive/MyDrive/L1/Notebooks/dataset/malgun.ttf"
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# 14년과 15년 가을데이터 READ
fall14 = pd.read_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/FALL14.pkl')
fall15 = pd.read_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/FALL15.pkl')

In [4]:
fall14

,RECEIPTNO,CUSTNO,PURCHDATE,PURCHTIME,AFFIL,CLASSCODEL,CLASSCODEM,CLASSCODES,STORENO,GENDER,AGEGROUP,RESIDENCE,PURCHAMOUNT
0,08068631,07122,2014-09-06,17,B,8,0802,B080203,028,M,40세~44세,100,14000
1,08942410,17967,2014-11-30,15,B,15,1502,B150203,050,M,20세~24세,100,1750
2,08942410,17967,2014-11-30,15,B,47,4703,B470302,050,M,20세~24세,100,7900
3,08942410,17967,2014-11-30,15,B,77,7707,B770702,050,M,20세~24세,100,16898
4,08942410,17967,2014-11-30,15,B,82,8202,B820201,050,M,20세~24세,100,7731
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3470510,07638475,00599,2014-11-13,14,B,11,1109,B110902,019,F,60세이상,100,2000
3470511,07638475,00599,2014-11-13,14,B,11,1110,B111002,019,F,60세이상,100,27000
3470512,07638475,00599,2014-11-13,14,B,12,1206,B120605,019,F,60세이상,100,29940
3470513,07638475,00599,2014-11-13,14,B,38,3804,B380403,019,F,60세이상,100,3980


# 재구매율 낮은 제품 판매금액 제거

- 재구매 데이터제거 위해 전처리, 파생변수 생성

In [5]:
# 6가지의 제품군으로 재분류한 결과
# 각각 food(음식), nor(생활용품), cloth(의류), hobby(취미), rich(사치), etc(기타)
# 에 해당하는 코드 (제휴사 + 대분류코드)

total_food = ['A1','B1', 'B2', 'B3', 'B4', 'B5', 'B6',
          'B7', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16','B37', 'B38',
          'B43', 'B46', 'B47', 'B48','B52', 'B53', 'B54', 'B55',
          'B56', 'B57', 'B58', 'B59', 'B60', 'B61', 'B62', 'B63',
          'B64', 'B65', 'B67', 'B72','B81', 'B82', 'B83', 'B89', 'B91', 'B92',
         'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
         'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'D4',
        'B73','B74','B75','B76','B77','B78','B79','B80']
# B73에서 B80누락됐던 부분 식품에 포함 

total_nor = ['A2', 'A3', 'A7', 'A9', 'B17',
        'B18', 'B19', 'B20', 'B21', 'B22', 'B23', 'B24', 'B31',
         'B34', 'B66', 'B77', 'B84', 'B86', 'B87', 'B88',
             'C16', 'C17', 'D1', 'D2', 'D3', 'D5', 'D6']

total_cloth = ['A4', 'A5', 'B29','B30', 'B32', 'B33', 'B44',
               'B49', 'B50', 'B51', 'B68', 'B69']

total_hobby = ['A6', 'B25', 'B26', 'B27', 'B35', 'B36', 'B71']

total_rich = ['A8', 'B28', 'B70']
total_etc = ['B8', 'B45', 'B90', 'D7', 'D8']


# 위에서 분류한 것을 기준으로 mapping 진행
def prod_cat(data):
    if data in total_food:
        return 'food'
    elif data in total_nor:
        return 'nor'
    elif data in total_cloth:
        return 'cloth'
    elif data in total_hobby:
        return 'hobby'
    elif data in total_rich:
        return 'luxury'
    elif data in total_etc:
        return 'etc'

In [6]:
# 14,15년 데이터에 'AFFIL_CLASS'와 'prod_cat' column 추가

fall14['CLASSCODEL'] = fall14['CLASSCODEL'].astype(str)
fall15['CLASSCODEL'] = fall15['CLASSCODEL'].astype(str)
fall14['AFFIL_CLASS'] = fall14.apply(lambda x: x['AFFIL'] + x['CLASSCODEL'], axis=1)
fall15['AFFIL_CLASS'] = fall15.apply(lambda x: x['AFFIL'] + x['CLASSCODEL'], axis=1)

# 제휴사(AFFIL) 와 대분류코드(CLASS)를 합친 column인 AFFIL_CLASS 를 바탕으로
# 새로운 제품분류 column인 prod_cat column 생성
fall14['prod_cat'] = fall14.AFFIL_CLASS.map(prod_cat)
fall15['prod_cat'] = fall15.AFFIL_CLASS.map(prod_cat)

- 재구매율 0% 제품 제거

In [7]:
# 재구매율 0%인 제품 제거
fall14 = fall14.drop(fall14.loc[(fall14.AFFIL_CLASS=='B34')|
                                (fall14.AFFIL_CLASS=='B62')|
                               (fall14.AFFIL_CLASS=='B59')|
                               (fall14.AFFIL_CLASS=='B61')|
                               (fall14.AFFIL_CLASS=='B58')|
                               (fall14.AFFIL_CLASS=='B60')|
                               (fall14.AFFIL_CLASS=='B42')|
                               (fall14.AFFIL_CLASS=='B40')|
                               (fall14.AFFIL_CLASS=='B41')|
                               (fall14.AFFIL_CLASS=='B85')].index, axis=0)

fall15 = fall15.drop(fall15.loc[(fall15.AFFIL_CLASS=='B34')|
                                (fall15.AFFIL_CLASS=='B62')|
                               (fall15.AFFIL_CLASS=='B59')|
                               (fall15.AFFIL_CLASS=='B61')|
                               (fall15.AFFIL_CLASS=='B58')|
                               (fall15.AFFIL_CLASS=='B60')|
                               (fall15.AFFIL_CLASS=='B42')|
                               (fall15.AFFIL_CLASS=='B40')|
                               (fall15.AFFIL_CLASS=='B41')|
                               (fall15.AFFIL_CLASS=='B85')].index, axis=0)

# inflation 적용

- inflation 적용

In [8]:
cloth_hobby14 = fall14[(fall14.prod_cat == 'cloth')| (fall14.prod_cat =='hobby')]
cloth_hobby15 = fall15[(fall15.prod_cat == 'cloth')| (fall15.prod_cat =='hobby')]


In [9]:
# 의류 -0.4%, 취미 1.6% 적용
# 의류와 취미를 제외한 상품군은 0.5퍼센트로 적용 (일단 차이를 보기위해)

fall15['PURCHAMOUNT_scaled'] = fall15.apply(lambda x: x['PURCHAMOUNT'] * 0.9930 if x['prod_cat']=='food'
                                        else x['PURCHAMOUNT'] * 0.9930 if x['prod_cat']=='nor'
                                        else x['PURCHAMOUNT'] * 1.0042 if x['prod_cat']=='cloth'
                                        else x['PURCHAMOUNT'] * 0.9843 if x['prod_cat']=='hobby'
                                        else x['PURCHAMOUNT'] * 0.9930 if x['prod_cat']=='luxury'
                                        else x['PURCHAMOUNT'] * 0.9930, axis= 1) # etc
fall15['PURCHAMOUNT_scaled'] =  fall15['PURCHAMOUNT_scaled'].astype(float)


In [10]:
# 14년 데이터셋에서 고객번호와 고객당 총 구매액만 컬럼화

f14 = cloth_hobby14.groupby('CUSTNO').sum().reset_index()[['CUSTNO', 'PURCHAMOUNT']]
f14.rename(columns = {'PURCHAMOUNT':'14_purchases'}, inplace = True)

# 15년 데이터셋에서 고객번호와 고객당 총 구매액만 컬럼화# 15년 데이터셋에서 고객번호와 고객당 총 구매액만 컬럼화

f15 = fall15.groupby('CUSTNO').sum().reset_index()[['CUSTNO', 'PURCHAMOUNT_scaled']]
f15.rename(columns = {'PURCHAMOUNT_scaled':'15_purchases_scaled'}, inplace= True)
f15.head()

,CUSTNO,15_purchases_scaled
0,00001,6.561721e+06
1,00002,1.408782e+07
2,00003,3.668221e+05
3,00004,1.777055e+06
4,00005,1.724459e+06


# 의류와 취미 제품군에서 감소한 고객만 확인

- cloth와 hobby만 포함한 14,15년도 fall Dataframe 생성

In [11]:
cloth_hobby14 = fall14[(fall14.prod_cat == 'cloth')| (fall14.prod_cat =='hobby')]
cloth_hobby15 = fall15[(fall15.prod_cat == 'cloth')| (fall15.prod_cat =='hobby')]


In [12]:
group_cloth_hobby14 = cloth_hobby14.groupby('CUSTNO').sum()
group_cloth_hobby14.drop('PURCHTIME', axis=1, inplace=True)
group_cloth_hobby14.reset_index(inplace=True)
group_cloth_hobby14


group_cloth_hobby15 = cloth_hobby15.groupby('CUSTNO').sum()
group_cloth_hobby15.drop('PURCHTIME', axis=1, inplace=True)
group_cloth_hobby15.reset_index(inplace=True)
group_cloth_hobby15


cloth_hobby_total = pd.merge(group_cloth_hobby14, group_cloth_hobby15, on='CUSTNO', how='left')
cloth_hobby_total = cloth_hobby_total.fillna(0)
cloth_hobby_total


cloth_hobby_total['diff'] = cloth_hobby_total['PURCHAMOUNT_x'] - cloth_hobby_total['PURCHAMOUNT_y']


def spent_less(data):
    if data>=0: return 1
    elif data<0: return 0
    
cloth_hobby_total['spent_diff'] = cloth_hobby_total['diff'].map(spent_less)
cloth_hobby_total

,CUSTNO,PURCHAMOUNT_x,PURCHAMOUNT_y,PURCHAMOUNT_scaled,diff,spent_diff
0,00001,4860810,4598340.0,4.615214e+06,262470.0,1
1,00002,6558370,6926570.0,6.945018e+06,-368200.0,0
2,00004,1230170,890980.0,8.934386e+05,339190.0,1
3,00006,606500,1822200.0,1.829077e+06,-1215700.0,0
4,00007,7204940,12950400.0,1.300374e+07,-5745460.0,0
...,...,...,...,...,...,...
16768,19369,243600,391700.0,3.933451e+05,-148100.0,0
16769,19371,148000,0.0,0.000000e+00,148000.0,1
16770,19372,3638110,2036190.0,2.037057e+06,1601920.0,1
16771,19373,184900,98900.0,9.931538e+04,86000.0,1


In [13]:
# 구매감소한 사람과 아닌사람의 숫자 비교
ori_cust = cloth_hobby_total.rename(columns = {'spent_diff':'spent_less', }).drop(['PURCHAMOUNT_x','PURCHAMOUNT_y','PURCHAMOUNT_scaled','diff'], axis=1).copy()
ori_cust.spent_less.value_counts()

1    9049
0    7724
Name: spent_less, dtype: int64

In [14]:
ori_cust

,CUSTNO,spent_less
0,00001,1
1,00002,0
2,00004,1
3,00006,0
4,00007,0
...,...,...
16768,19369,0
16769,19371,1
16770,19372,1
16771,19373,1


In [15]:
ori_cust[['CUSTNO', 'spent_less']].to_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/ori_cust.pkl')

In [16]:
# 제품군을 groupby 하여 묶은 후 2014년의 제품군 별 총 매출액 계산
purch_by_prodcat = fall14.groupby('prod_cat').sum()[['PURCHAMOUNT']]
# 제품군을 groupby 하여 묶은 후 2015년의 제품군 별 총 매출액 계산
purch_by_prodcat15 = fall15.groupby('prod_cat').sum()[['PURCHAMOUNT']]

In [17]:
# 15년의 제품군 별 총액을 _14에 15Purch라는 새로운 column을 생성하여 저장
purch_by_prodcat['15_purchases'] = purch_by_prodcat15.PURCHAMOUNT
purch_by_prodcat.rename(columns = {'PURCHAMOUNT':'14_purchases', }, inplace = True)
purch_by_prodcat

,14_purchases,15_purchases
prod_cat,,
cloth,28269908870,27806480670
etc,982547076,1106168320
food,21067667723,23783732682
hobby,7423925940,7258360920
luxury,7987014700,9536549190
nor,23970841504,24650700801


In [18]:
purch_by_prodcat.to_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/purch_by_prodcat.pkl')

In [19]:
# 각 제품별 share 확인하는 '15_purch_share(%)' column 생성
purch_by_prodcat.insert(2, '15_purch_share(%)', purch_by_prodcat.apply(lambda x: 
                        round(x['15_purchases']/purch_by_prodcat['15_purchases'].sum()*100,2).sum(), axis=1))

# 14년의 제품군 별 총 금액과 15년의 제품군 별 총 금액의 차이를 계산하여
# diff 라는 새로운 column에 저장
purch_by_prodcat['diff'] = \
purch_by_prodcat.apply(lambda x: round(x['15_purchases'] - x['14_purchases']), axis=1)

# 차이가 얼마나 나왔는지 퍼센트로 계산
purch_by_prodcat['change(%)'] =\
purch_by_prodcat.apply(lambda x: 
                        round(x['diff']/x['15_purchases']*100,2).sum(), axis=1)

purch_by_prodcat

,14_purchases,15_purchases,15_purch_share(%),diff,change(%)
prod_cat,,,,,
cloth,28269908870,27806480670,29.54,-463428200,-1.67
etc,982547076,1106168320,1.17,123621244,11.18
food,21067667723,23783732682,25.26,2716064959,11.42
hobby,7423925940,7258360920,7.71,-165565020,-2.28
luxury,7987014700,9536549190,10.13,1549534490,16.25
nor,23970841504,24650700801,26.18,679859297,2.76


In [20]:
fall1415 = pd.read_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/fall1415.pkl')

In [21]:
# 일단 가을 데이터 전부에서 고객별로 라벨값 추가
full_data_with_label = fall1415.join(ori_cust.set_index('CUSTNO')['spent_less'], 
                                     on='CUSTNO', how='right')


In [22]:
full_data_with_label.spent_less.unique()

array([1, 0])

In [23]:
full_data_with_label

,RECEIPTNO,CUSTNO,PURCHDATE,PURCHTIME,AFFIL,CLASSCODEL,CLASSCODEM,CLASSCODES,STORENO,GENDER,AGEGROUP,RESIDENCE,PURCHAMOUNT,Year,AFFIL_CLASS,spent_less
167386,10972931,00001,2014-10-09,15,C,3,0304,C030404,318,M,60세이상,060,1990,2014,C3,1
167387,10972931,00001,2014-10-09,15,C,7,0701,C070102,318,M,60세이상,060,3980,2014,C7,1
167388,10972931,00001,2014-10-09,15,C,7,0704,C070402,318,M,60세이상,060,8400,2014,C7,1
167389,10972931,00001,2014-10-09,15,C,8,0802,C080201,318,M,60세이상,060,3000,2014,C8,1
170493,10972931,00001,2014-10-09,15,C,1,0101,C010101,318,M,60세이상,060,9900,2014,C1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3337949,07296382,19375,2015-11-20,15,B,2,0201,B020102,016,F,25세~29세,049,20000,2015,B2,0
3362190,07297104,19375,2015-11-30,23,B,10,1008,B100802,016,F,25세~29세,049,3450,2015,B10,0
3362191,07297104,19375,2015-11-30,23,B,14,1401,B140102,016,F,25세~29세,049,2980,2015,B14,0
3362192,07297104,19375,2015-11-30,23,B,14,1401,B140103,016,F,25세~29세,049,970,2015,B14,0


In [25]:
# 전체 데이터 피클로 저장

full_data_with_label.to_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/full_data_with_label.pkl')

In [26]:
full_data_with_label.groupby('CUSTNO').count()

,RECEIPTNO,PURCHDATE,PURCHTIME,AFFIL,CLASSCODEL,CLASSCODEM,CLASSCODES,STORENO,GENDER,AGEGROUP,RESIDENCE,PURCHAMOUNT,Year,AFFIL_CLASS,spent_less
CUSTNO,,,,,,,,,,,,,,,
00001,228,228,228,228,228,228,228,228,228,228,228,228,228,228,228
00002,240,240,240,240,240,240,240,240,240,240,240,240,240,240,240
00004,168,168,168,168,168,168,168,168,168,168,168,168,168,168,168
00006,352,352,352,352,352,352,352,352,352,352,352,352,352,352,352
00007,207,207,207,207,207,207,207,207,207,207,207,207,207,207,207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19369,491,491,491,491,491,491,491,491,491,491,491,491,491,491,491
19371,355,355,355,355,355,355,355,355,355,355,355,355,355,355,355
19372,239,239,239,239,239,239,239,239,239,239,239,239,239,239,239


In [27]:
tdf = full_data_with_label.copy()

In [28]:
# 제휴사 구분 없이 분류
tdf['AFFIL_CLASS_2'] = tdf.apply(lambda x: x['AFFIL'] + x['CLASSCODEM'], axis=1)

tdf['cloth_cat'] = \
tdf.apply(lambda x: 'cloth_men' if x['AFFIL_CLASS_2'] in ['A0501','A0506','B3004','B3005','B3006','B5002']
                else 'cloth_women' if x['AFFIL_CLASS_2'] in ['A0407','B4903','B4905','B4906','B4901']
                else 'cloth_kids' if x['AFFIL_CLASS_2'] in ['A0504','B3101','B3205','B3206','B3307','B3204']
                else 'cloth_sports' if x['AFFIL_CLASS_2'] in ['B3203','B5102','B5103','B3501','B3504','B3505','B3507','B3608','B5003','B5007']
                else 'cloth_lingerie' if x['AFFIL_CLASS_2'] in ['A0406','B3301','B3302','B3306','B3308','B6808','B3304']
                else 'cloth_casual' if x['AFFIL_CLASS_2'] in ['A0402','A0405','A0505','A0502','B2903','B3002','B3003',\
                                                              'B5004','B5005','B2904','B4902','B4904']
                else 'cloth_business' if x['AFFIL_CLASS_2'] in ['B3001', 'A0403']
                else 'cloth_etc' if x['AFFIL_CLASS_2'] in ['A0409','A0410','A0408','A0503','A0404','B2901','B2902','B3102','B4504']
                else 'uncategorized', axis= 1)
tdf['hobby_cat'] = \
tdf.apply(lambda x: 'hobby_media' if x['AFFIL_CLASS_2'] in ['B2501', 'B2502', 
                                                      'B2503', 'B2504', 'B2505', 'B2506',
                                                      'B2603', 'B2604', 'B2607', 'B2608']
                else 'hobby_child' if x['AFFIL_CLASS_2'] in ['B2602', 'B2605', 'B2609', 'B2610']
                else 'hobby_pet' if x['AFFIL_CLASS_2'] in ['B2701', 'B2702', 'B2703', 'B2705', 'B2706', 'B2707']
                else 'hobby_outdoor' if x['AFFIL_CLASS_2'] in ['B7102', 'B7103', 'B3501', 'B3502',
                                                        'B3503', 'B3505', 'B3506', 'B3604', 
                                                         'B2606', 'B5105', 'B3605']
                else 'hobby_sports' if x['AFFIL_CLASS_2'] in ['B3504', 'B3507', 'B3608']
                else 'hobby_health' if x['AFFIL_CLASS_2'] in ['B2704', 'B3607', 'B3603']
                else 'hobby_instex' if x['AFFIL_CLASS_2'] in ['B3602', 'B3601', 'B3606']
                else 'hobby_golf' if x['AFFIL_CLASS_2'] in ['A0602']
                else 'uncategorized', axis= 1)

In [29]:
tdf = tdf.set_index('CUSTNO')

In [30]:
tdf1 = tdf[(tdf.cloth_cat != 'uncategorized')|(tdf.hobby_cat != 'uncategorized')]
tdf1.reset_index()

,CUSTNO,RECEIPTNO,PURCHDATE,PURCHTIME,AFFIL,CLASSCODEL,CLASSCODEM,CLASSCODES,STORENO,GENDER,AGEGROUP,RESIDENCE,PURCHAMOUNT,Year,AFFIL_CLASS,spent_less,AFFIL_CLASS_2,cloth_cat,hobby_cat
0,00001,02857006,2014-10-05,17,A,5,0506,A050601,012,M,60세이상,060,342400,2014,A5,1,A0506,cloth_men,uncategorized
1,00001,02857007,2014-10-05,17,A,5,0506,A050601,012,M,60세이상,060,78000,2014,A5,1,A0506,cloth_men,uncategorized
2,00001,02857020,2014-10-05,17,A,5,0501,A050113,012,M,60세이상,060,500000,2014,A5,1,A0501,cloth_men,uncategorized
3,00001,02947490,2014-10-19,12,A,4,0402,A040222,012,M,60세이상,060,314640,2014,A4,1,A0402,cloth_casual,uncategorized
4,00001,02947512,2014-10-19,13,A,5,0503,A050306,012,M,60세이상,060,406980,2014,A5,1,A0503,cloth_etc,uncategorized
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339012,19373,02844688,2014-10-03,20,A,4,0402,A040228,042,F,25세~29세,100,69000,2014,A4,1,A0402,cloth_casual,uncategorized
339013,19373,05366898,2015-10-25,14,A,4,0402,A040217,010,F,25세~29세,100,29000,2015,A4,1,A0402,cloth_casual,uncategorized
339014,19373,05242502,2015-10-04,15,A,4,0402,A040217,042,F,25세~29세,100,69900,2015,A4,1,A0402,cloth_casual,uncategorized
339015,19375,07268767,2014-11-14,22,B,33,3304,B330403,016,F,25세~29세,049,12800,2014,B33,0,B3304,cloth_lingerie,uncategorized


In [31]:
tdf1.groupby('CUSTNO').count()

,RECEIPTNO,PURCHDATE,PURCHTIME,AFFIL,CLASSCODEL,CLASSCODEM,CLASSCODES,STORENO,GENDER,AGEGROUP,RESIDENCE,PURCHAMOUNT,Year,AFFIL_CLASS,spent_less,AFFIL_CLASS_2,cloth_cat,hobby_cat
CUSTNO,,,,,,,,,,,,,,,,,,
00001,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45
00002,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39
00004,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18
00006,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39
00007,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19369,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17
19371,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
19372,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46


In [32]:
tdf2 = tdf1.reset_index()
tdf2.to_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/full_data_with_label_FINAL.pkl')

In [33]:
temp = tdf2[['CUSTNO', 'spent_less','Year']]
a_2014 = temp[temp.Year ==2014] 
a_2015 = temp[temp.Year ==2015] #2014,2015년 비교
temp2 = pd.merge(a_2014,a_2015 , how='left',left_on = a_2014.CUSTNO,right_on = a_2015.CUSTNO) #2014,2015 left결합
temp2.drop_duplicates(inplace=True)
custno_with_label = temp2.copy()[['CUSTNO_x','spent_less_x']].reset_index().drop('index',axis=1)
custno_with_label.rename(columns = {'CUSTNO_x':'CUSTNO', 'spent_less_x':'spent_less'}, inplace = True)
custno_with_label.to_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/custno_only.pkl')
custno_with_label


,CUSTNO,spent_less
0,00001,1
1,00002,0
2,00004,1
3,00006,0
4,00007,0
...,...,...
16290,19369,0
16291,19371,1
16292,19372,1
16293,19373,1


In [34]:
custno_with_label.spent_less.value_counts()

1    8818
0    7477
Name: spent_less, dtype: int64

In [35]:
df = pd.read_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/full_data_with_label_FINAL.pkl')
df.groupby('CUSTNO').count()

,RECEIPTNO,PURCHDATE,PURCHTIME,AFFIL,CLASSCODEL,CLASSCODEM,CLASSCODES,STORENO,GENDER,AGEGROUP,RESIDENCE,PURCHAMOUNT,Year,AFFIL_CLASS,spent_less,AFFIL_CLASS_2,cloth_cat,hobby_cat
CUSTNO,,,,,,,,,,,,,,,,,,
00001,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45
00002,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39
00004,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18
00006,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39
00007,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19369,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17
19371,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
19372,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46


In [36]:
df

,CUSTNO,RECEIPTNO,PURCHDATE,PURCHTIME,AFFIL,CLASSCODEL,CLASSCODEM,CLASSCODES,STORENO,GENDER,AGEGROUP,RESIDENCE,PURCHAMOUNT,Year,AFFIL_CLASS,spent_less,AFFIL_CLASS_2,cloth_cat,hobby_cat
0,00001,02857006,2014-10-05,17,A,5,0506,A050601,012,M,60세이상,060,342400,2014,A5,1,A0506,cloth_men,uncategorized
1,00001,02857007,2014-10-05,17,A,5,0506,A050601,012,M,60세이상,060,78000,2014,A5,1,A0506,cloth_men,uncategorized
2,00001,02857020,2014-10-05,17,A,5,0501,A050113,012,M,60세이상,060,500000,2014,A5,1,A0501,cloth_men,uncategorized
3,00001,02947490,2014-10-19,12,A,4,0402,A040222,012,M,60세이상,060,314640,2014,A4,1,A0402,cloth_casual,uncategorized
4,00001,02947512,2014-10-19,13,A,5,0503,A050306,012,M,60세이상,060,406980,2014,A5,1,A0503,cloth_etc,uncategorized
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339012,19373,02844688,2014-10-03,20,A,4,0402,A040228,042,F,25세~29세,100,69000,2014,A4,1,A0402,cloth_casual,uncategorized
339013,19373,05366898,2015-10-25,14,A,4,0402,A040217,010,F,25세~29세,100,29000,2015,A4,1,A0402,cloth_casual,uncategorized
339014,19373,05242502,2015-10-04,15,A,4,0402,A040217,042,F,25세~29세,100,69900,2015,A4,1,A0402,cloth_casual,uncategorized
339015,19375,07268767,2014-11-14,22,B,33,3304,B330403,016,F,25세~29세,049,12800,2014,B33,0,B3304,cloth_lingerie,uncategorized


In [37]:
tttt = pd.merge(custno_with_label,df, how='inner', on='CUSTNO')
tttt.to_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/full_data_with_label_FINAL.pkl')

In [38]:
tttt

,CUSTNO,spent_less_x,RECEIPTNO,PURCHDATE,PURCHTIME,AFFIL,CLASSCODEL,CLASSCODEM,CLASSCODES,STORENO,GENDER,AGEGROUP,RESIDENCE,PURCHAMOUNT,Year,AFFIL_CLASS,spent_less_y,AFFIL_CLASS_2,cloth_cat,hobby_cat
0,00001,1,02857006,2014-10-05,17,A,5,0506,A050601,012,M,60세이상,060,342400,2014,A5,1,A0506,cloth_men,uncategorized
1,00001,1,02857007,2014-10-05,17,A,5,0506,A050601,012,M,60세이상,060,78000,2014,A5,1,A0506,cloth_men,uncategorized
2,00001,1,02857020,2014-10-05,17,A,5,0501,A050113,012,M,60세이상,060,500000,2014,A5,1,A0501,cloth_men,uncategorized
3,00001,1,02947490,2014-10-19,12,A,4,0402,A040222,012,M,60세이상,060,314640,2014,A4,1,A0402,cloth_casual,uncategorized
4,00001,1,02947512,2014-10-19,13,A,5,0503,A050306,012,M,60세이상,060,406980,2014,A5,1,A0503,cloth_etc,uncategorized
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337918,19373,1,02844688,2014-10-03,20,A,4,0402,A040228,042,F,25세~29세,100,69000,2014,A4,1,A0402,cloth_casual,uncategorized
337919,19373,1,05366898,2015-10-25,14,A,4,0402,A040217,010,F,25세~29세,100,29000,2015,A4,1,A0402,cloth_casual,uncategorized
337920,19373,1,05242502,2015-10-04,15,A,4,0402,A040217,042,F,25세~29세,100,69900,2015,A4,1,A0402,cloth_casual,uncategorized
337921,19375,0,07268767,2014-11-14,22,B,33,3304,B330403,016,F,25세~29세,049,12800,2014,B33,0,B3304,cloth_lingerie,uncategorized


In [39]:
temp = df.copy()
a_2014 = temp[temp.Year ==2014] 
a_2015 = temp[temp.Year ==2015] #2014,2015년 비교
temp2 = pd.merge(a_2014,a_2015 , how='left',left_on = a_2014.CUSTNO,right_on = a_2015.CUSTNO) #2014,2015 left결합
temp2.drop_duplicates(inplace=True)
custno_with_label = temp2.copy()[['CUSTNO_x','spent_less_x']].reset_index().drop('index',axis=1)
custno_with_label.rename(columns = {'CUSTNO_x':'CUSTNO', 'spent_less_x':'spent_less'}, inplace = True)
custno_with_label

,CUSTNO,spent_less
0,00001,1
1,00001,1
2,00001,1
3,00001,1
4,00001,1
...,...,...
2709585,19373,1
2709586,19373,1
2709587,19373,1
2709588,19373,1
